In [26]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np

In [16]:
reference_dataset = r"D:\projects\sovereign-risk\Thailand\data\exposure\CISI_THA.tif"
flood_dataset = r"D:\projects\sovereign-risk\Thailand\data\flood\maps\THA_global_pc_h100glob.tif"
output_dataset = r"D:\projects\sovereign-risk\Thailand\data\flood\maps\aggregated_0p1_degree\test_100.tif"

In [19]:
# Load the high-resolution flood map
with rasterio.open(flood_dataset) as high_res:
    high_res_data = high_res.read(1)  # Read the first band
    # Replace NoData values with 0
    nodata_value = high_res.nodata
    if nodata_value is not None:
        high_res_data[np.isnan(high_res_data)] = 0
        high_res_data[high_res_data == nodata_value] = 0
    
    high_res_transform = high_res.transform

In [20]:
# Load the coarse resolution dataset to use as a reference
with rasterio.open(reference_dataset) as coarse_res:
    coarse_res_transform = coarse_res.transform
    coarse_res_width = coarse_res.width
    coarse_res_height = coarse_res.height

In [21]:
# Calculate the scaling factors
scale_x = coarse_res_transform.a / high_res_transform.a
scale_y = coarse_res_transform.e / high_res_transform.e

In [27]:
# Use rasterio's reproject to resample and aggregate the high-resolution data
with rasterio.open(flood_dataset) as src:
    # Define the metadata for the new aggregated dataset
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": coarse_res_height,
        "width": coarse_res_width,
        "transform": coarse_res_transform
    })
    with rasterio.open(output_dataset, 'w', **out_meta) as dst:
    # Perform the aggregation
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=coarse_res_transform,
                dst_crs=coarse_res.crs,
                resampling=Resampling.average
            )